In [63]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from keras.layers import Input, Dense, Lambda, Flatten, Reshape
from keras.layers import Conv1D
from keras.models import Model
from keras import backend as K

n_features = 4 
labels = pd.read_csv("Data/Boat_nominal_data/Boat_mix_len_labels.csv")
labels = labels.drop(columns="Unnamed: 0") 
labels = np.array(labels)
max_len = 0


def prepare_training(path, n_runs):
    def get_max_len(sequence_list):
        m_len = 0
        for seq in sequence_list:
            if len(seq) > m_len:
                m_len = len(seq)
        max_len = m_len
        print(max_len)
        return m_len
    
    
    def construct_matrix(sequence_list):
        max_len = get_max_len(sequence_list)
        train_matrix = np.zeros(shape=(n_runs, max_len, n_features))
        for index,run in enumerate(sequence_list):
            train_matrix[index, :len(run)] = run
        print(train_matrix.shape)
        return train_matrix
        
        
    def stadard_sequences(sequences):
        for i, seq in enumerate(sequences):
            sequences[i] = MinMaxScaler(feature_range=[0, 1]).fit_transform(seq)
        return sequences       
    
    def read_sequences():
        run_list_mix = []
        for index in range(n_runs):
            run_csv = pd.read_csv(path+str(index))
            run_csv = run_csv.drop(columns=['Unnamed: 0'])
            run_list_mix.append(run_csv)
        stand_sequences = stadard_sequences(run_list_mix)
        padded_matrix = construct_matrix(stand_sequences)
        return padded_matrix
    
    return read_sequences()
    

train_matrix = prepare_training("Mix_sequences_var_length/run^", n_runs=300) 


180
(300, 180, 4)


In [64]:

from keras.layers import Input, LSTM, RepeatVector, Conv2DTranspose
from keras.losses import mse
from keras.models import Model


filters = 50
intermediate_dimension = 30 
latent_dim = 10

def Conv1DTranspose(input_tensor, filters, kernel_size,last, strides=2, padding='same'):
        if last:
            activation = 'linear'
        else:
            activation = 'relu'
        x = Lambda(lambda x: K.expand_dims(x, axis=2))(input_tensor)
        x = Conv2DTranspose(filters=filters, kernel_size=(kernel_size, 1), 
                            strides=(strides, 1), padding=padding,
                            activation=activation)(x)
        x = Lambda(lambda x: K.squeeze(x, axis=2))(x)
        return x


def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon


def repeat(x):
    steps_matrix = K.ones_like(x[0][:, :, :1])
    latent_matrix = K.expand_dims(x[1], axis=1)
    return K.batch_dot(steps_matrix, latent_matrix)


def create_vae():
    print(max_len)
    inputs = Input(shape=(180, n_features))
    x = inputs
    
    for i in range(2):
        x = Conv1D(filters=filters,
                   kernel_size=2,
                   strides=2,
                   padding='same')(x)
    
    shape = K.int_shape(x)
    
    #before_flattening = LSTM(intermediate_dimension, return_sequences=True)(x)
    #x_flat = LSTM(intermediate_dimension)(before_flattening)
    x = Flatten()(x)
    
    
    embeddings = Dense(latent_dim)(x)
    
    z_mean = Dense(latent_dim, name='z_mean',)(embeddings)
    z_log_var = Dense(latent_dim, name='z_log_var')(embeddings)
    z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])
    
    encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
    
    latent_inputs = Input(shape=(latent_dim,), name='latent_inputs')
    #x = Lambda(repeat)([before_flattening, z])
    x = Dense(shape[1]*shape[2])(latent_inputs)
    x = Reshape((shape[1],shape[2]))(x)
    
    
    print(K.int_shape(x))
    for i in range(2):
        x = Conv1DTranspose(input_tensor=x,
                            filters=filters,
                            kernel_size=2,
                            last=False,
                            padding='same')
    
    
    outputs = Conv1DTranspose(input_tensor=x,
                              filters=n_features,
                              kernel_size=2,
                              strides=1,
                              last=True,
                              padding='same')
    
    encoder.summary()
    decoder = Model(latent_inputs, outputs)
    decoder.summary()
    outputs = decoder(encoder.outputs[2])
    reconstruction_loss = mse(K.flatten(inputs), K.flatten(outputs))
    #reconstruction_loss *= sequence_length*4
    kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), 
                             axis=-1)
    loss = K.mean(reconstruction_loss+0*kl_loss)
    vae = Model(inputs, outputs, name='vae')
    vae.add_loss(loss)
    #vae.summary()
    vae.compile(optimizer='adam')
    return vae, encoder


def create_ae():
    inputs = Input(shape=(900, n_features))
    x = inputs
    for i in range(2):
        x = Conv1D(filters=filters, kernel_size=2, strides=2,
                   activation='relu', padding='same')(x)
    shape = K.int_shape(x)   
    before_flattening = x #LSTM(filters, return_sequences=True)(x)   
    #encoded = LSTM(intermediate_dimension)(before_flattening)
    encoded = Flatten()(x)
    encoded = Dense(latent_dim)(encoded)
    #x = Lambda(repeat)([before_flattening, encoded])
    
    latent_inputs = Input(shape=(latent_dim,), name='latent_inputs')
    x = Dense(shape[1]*shape[2])(latent_inputs)
    x = Reshape((shape[1],shape[2]))(x)
    
    for i in range(2):
        x = Conv1DTranspose(input_tensor=x, filters=filters,
                            kernel_size=2, padding='same', last=False)
       
       
    #decoded = LSTM(n_features, return_sequences=True)(x)
    
    output = Dense(n_features)(x)
    
    encoder = Model(inputs, encoded)
    decoder = Model(latent_inputs, output)
    
    output = decoder(encoder.output)
    sequence_autoencoder = Model(inputs, output)
    sequence_autoencoder.summary()
    sequence_autoencoder.compile(optimizer='adam', loss='mse')
    return sequence_autoencoder, encoder


model, encoder = create_vae()


0
(None, 45, 50)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 180, 4)       0                                            
__________________________________________________________________________________________________
conv1d_11 (Conv1D)              (None, 90, 50)       450         input_6[0][0]                    
__________________________________________________________________________________________________
conv1d_12 (Conv1D)              (None, 45, 50)       5050        conv1d_11[0][0]                  
__________________________________________________________________________________________________
flatten_6 (Flatten)             (None, 2250)         0           conv1d_12[0][0]                  
__________________________________________________________________________________________________
dense_11 (

In [65]:
from keras.callbacks import ModelCheckpoint


def train():
    print(train_matrix.shape)
    model.fit(train_matrix, epochs=100, verbose=1)
    model.save_weights("Models/Weights/VAE_CONV_BATCH_300_ONLY_NOM_MATRIX_LEN.hdf5")


train()


(300, 180, 4)


Epoch 1/100


 32/300 [==>...........................] - ETA: 31s - loss: 0.3258

 64/300 [=====>........................] - ETA: 16s - loss: 0.3220

128/300 [===========>..................] - ETA: 6s - loss: 0.3084 

...

192/300 [==================>...........] - ETA: 0s - loss: 7.3481e-05

224/300 [=====================>........] - ETA: 0s - loss: 7.2050e-05

256/300 [========================>.....] - ETA: 0s - loss: 7.2218e-05

288/300 [===========================>..] - ETA: 0s - loss: 7.2326e-05

300/300 [==============================] - 1s 3ms/step - loss: 7.1950e-05


In [9]:

model.load_weights("Models/Weights/VAE_CONV_BATCH_1000_diff_len_980_MATRIX_LEN.hdf5")


In [66]:

from sklearn.decomposition import PCA

encodings = encoder.predict(train_matrix)
enc_mean, enc_var, z_enc = encodings[0], encodings[1], encodings[2]
print(enc_mean.shape, enc_var.shape, z_enc.shape)


def return_mask(num, labs):
    arg = np.squeeze(np.argwhere(labs == num))
    return arg


masks = [return_mask(num, np.array(labels))[:, 0] for num in range(0, 9)]


(300, 10) (300, 10) (300, 10)


In [69]:

from mpl_toolkits.mplot3d import Axes3D  


def plot_pca(title, i): 
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    markers = ['o', 'o', 'o', 'o', '^', '^', '^', '^', '^', '^']
    for index, mask in enumerate(masks):
        ax.scatter(principalComponents[:, 0][mask], 
                   principalComponents[:, 1][mask],  
                   principalComponents[:, 2][mask], marker=markers[index])
    ax.scatter(unseen_encoding[0][:,0],
               unseen_encoding[0][:,1],
               unseen_encoding[0][:,2])
    plt.legend(labels=np.arange(0, 9))
    plt.title(str(title))
    plt.show()
    
    for mask in masks:
        plt.scatter(x=principalComponents[:, 0][mask], 
                    y=principalComponents[:, 1][mask],
                    alpha=0.5)
    for mask in unseen_mask:
        plt.scatter(unseen_encoding[0][:,0][mask],
               unseen_encoding[0][:,1][mask])
        #break
    
    plt.legend(labels=np.arange(0, 9))
    plt.title(str(title))
    plt.show()


enc_list = [enc_mean, enc_var, z_enc]
titles = ["MEAN","LOG_VAR","SAMPLED"]
for i,enc in enumerate(enc_list):
    scaler = StandardScaler()
    enc_input = scaler.fit_transform(enc) 
    pca = PCA(3)
    principalComponents = pca.fit_transform(enc_input)
    print(principalComponents.shape)
    print(pca.explained_variance_ratio_)
    plot_pca('Sequences'+titles[i], 0)
    
    principalComponents = enc
    plot_pca('Sequences_Not_Pca'+titles[i], 0)
    


(300, 3)
[0.7402665  0.12076205 0.07642484]


(300, 3)
[0.8694305  0.05360297 0.03483511]


(300, 3)
[0.2625509  0.11087869 0.10847034]


In [13]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=0)

tsne_obj= tsne.fit_transform(enc_input)
print(tsne_obj.shape)

for mask in masks:
    plt.scatter(x=tsne_obj[:, 0][mask], 
                y=tsne_obj[:, 1][mask],
                alpha=0.5)
plt.show()

(1000, 2)


In [67]:



unseen_labs = pd.read_csv("Data/Boat_nominal_data/Boat_unseen_labels_mix.csv")
unseen_labs = unseen_labs.drop(columns="Unnamed: 0") 
unseen_labs = np.array(unseen_labs)
unseen_mask = [return_mask(num, np.array(unseen_labs))[:, 0] for num in range(0, 9)]

unseen_sequences_matrix = prepare_training("Mix_sequences_var_length/run_unseen^", 
                                           n_runs=300)
print(unseen_sequences_matrix.shape)
unseen_encoding = encoder.predict(unseen_sequences_matrix)
print(len(unseen_encoding), unseen_encoding[0].shape)

#print(unseen_encoding[0][:, 0])
# def plot_unseen():
#     #CALLS PLOT_PCA with new points
    

180
(300, 180, 4)
(300, 180, 4)
3 (300, 10)
[4.4688973 4.3288627 4.315207  4.3204584 4.310945  4.296671  4.415869
 4.332097  4.3101068 4.3172507 4.31533   4.322782  4.308103  4.3069887
 4.315312  4.310229  4.3640795 4.309468  4.3096905 4.2712445 4.3155103
 4.3011675 4.3260026 4.437308  4.299027  4.31146   4.318935  4.3296742
 4.2441087 4.3153157 4.4240346 4.305037  4.352069  4.3299084 4.3045835
 4.3156347 4.359144  4.3127174 4.2427807 4.330794  4.3130198 4.3166895
 4.3180294 4.3170176 4.331828  4.3247414 4.4413037 4.3163085 4.5211115
 4.3292804 4.316901  4.319405  4.311852  4.155762  4.334037  4.321927
 4.1947527 4.3175983 4.3164735 4.444598  4.2287292 4.3253703 4.4867635
 4.2967114 4.311278  4.314728  4.313974  4.487346  4.300607  4.321642
 4.3215904 4.3027062 4.2947226 4.31821   4.314074  4.322484  4.3113317
 4.245809  4.3365846 4.2958155 4.319854  4.4150457 4.3779964 4.3193655
 4.3177586 4.366511  4.3063035 4.4080043 4.333152  4.3248363 4.3175664
 4.476707  4.3625827 4.302556  4.322